In [1]:
import pandas as pd
%pip install openpyxl
import re



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#criar base de dados unindo a aero_ANAC com rotaer_final_3

# Ler os arquivos CSV
aero_ANAC = pd.read_csv("output/aero_ANAC.csv", encoding="utf-8")
rotaer_final = pd.read_csv("output/base_rotaer_final.csv", encoding="utf-8")

# Renomear a coluna 'codigo_aeroporto' para 'codigo_oaci' no rotaer_final_3
rotaer_final.rename(columns={'codigo_aeroporto': 'codigo_oaci'}, inplace=True)

# Realizar o merge (join) das bases mantendo todos os dados de aero_ANAC
aero_ANAC_rotaer = aero_ANAC.merge(
    rotaer_final[['codigo_oaci', 'Auxilios_Pares', 'Auxilios_Individuais', 'RFFS']],  # Apenas as colunas que serão adicionadas
    on='codigo_oaci',
    how='left'  # Mantém todos os registros de aero_ANAC, adicionando os dados de base_rotaer_final onde houver correspondência
)

# Substituir valores NaN por 'N/A' para manter a formatação
aero_ANAC_rotaer.fillna("N/A", inplace=True)

# Exibir informações para conferência
print(f"Total de registros em aero_ANAC_rotaer: {len(aero_ANAC_rotaer)}")


# Salvar a base final corrigida
aero_ANAC_rotaer.to_csv("data/csv/aero_ANAC_rotaer.csv", index=False, encoding="utf-8")
print("Arquivo 'aero_ANAC_rotaer.csv' salvo com sucesso!")




Total de registros em aero_ANAC_rotaer: 4097
Arquivo 'aero_ANAC_rotaer.csv' salvo com sucesso!


In [3]:
#tirar _pista3

# Identifica as colunas que terminam com '_pista3'
colunas_para_excluir = [col for col in aero_ANAC_rotaer.columns if col.endswith('_pista3')]

# Remove essas colunas do DataFrame
aero_ANAC_rotaer = aero_ANAC_rotaer.drop(columns=colunas_para_excluir)


In [4]:
#adicionar nova base 
AIDS = pd.read_excel("data/csv/AIDS_aero.xlsx")

In [5]:
# Garante que os códigos OACI estão como string
AIDS['codigo_oaci'] = AIDS['codigo_oaci'].astype(str)
aero_ANAC_rotaer['codigo_oaci'] = aero_ANAC_rotaer['codigo_oaci'].astype(str)

# Cria uma cópia da base original
aero_rotaer_com_aids = aero_ANAC_rotaer.copy()

# Inicializa as colunas
aero_rotaer_com_aids['AIDS_pista1'] = None
aero_rotaer_com_aids['Aux_pista1'] = None
aero_rotaer_com_aids['AIDS_pista2'] = None
aero_rotaer_com_aids['Aux_pista2'] = None

# Loop pelas linhas do DataFrame AIDS
for _, row in AIDS.iterrows():
    cod = row['codigo_oaci']
    
    # Lista de cabeceiras e suas respectivas colunas de auxílio e AIDS
    entradas = [
        (row['C1_pista1'], row['AUX1_pista1'], row['AIDS1_pista1'], 'pista1'),
        (row['C2_pista1'], row['AUX2_pista1'], row['AIDS2_pista1'], 'pista1'),
        (row['C1_pista2'], row['AUX1_pista2'], row['AIDS1_pista2'], 'pista2'),
        (row['C2_pista2'], row['AUX2_pista2'], row['AIDS2_pista2'], 'pista2'),
    ]
    
    for cab, aux, aids, pista in entradas:
        if pd.isna(cab):
            continue
        
        cab_str = str(cab).zfill(2)
        aux_formatado = f"{aux} {cab_str}" if pd.notna(aux) else None
        aids_formatado = f"{aids} {cab_str}" if pd.notna(aids) else None

        # Encontrar linha correspondente no outro DataFrame
        filtro = aero_rotaer_com_aids['codigo_oaci'] == cod
        if filtro.any():
            idx = aero_rotaer_com_aids[filtro].index[0]

            designacao = str(aero_rotaer_com_aids.at[idx, f'designacao_{pista}'])
            cabeceiras = [x.strip() for x in designacao.split('/')] if pd.notna(designacao) else []

            if cab_str in cabeceiras:
                # Preenche ou acumula valores
                if aux_formatado:
                    if aero_rotaer_com_aids.at[idx, f'Aux_{pista}']:
                        aero_rotaer_com_aids.at[idx, f'Aux_{pista}'] += f" / {aux_formatado}"
                    else:
                        aero_rotaer_com_aids.at[idx, f'Aux_{pista}'] = aux_formatado

                if aids_formatado:
                    if aero_rotaer_com_aids.at[idx, f'AIDS_{pista}']:
                        aero_rotaer_com_aids.at[idx, f'AIDS_{pista}'] += f" / {aids_formatado}"
                    else:
                        aero_rotaer_com_aids.at[idx, f'AIDS_{pista}'] = aids_formatado


In [6]:
# Lista de todas as colunas atuais
colunas = list(aero_rotaer_com_aids.columns)

# Encontra as posições das colunas de designação
idx_p1 = colunas.index('designacao_pista1')
idx_p2 = colunas.index('designacao_pista2')

# Remove as colunas novas temporariamente da lista original (se estiverem no fim)
for c in ['AIDS_pista1', 'Aux_pista1', 'AIDS_pista2', 'Aux_pista2']:
    if c in colunas:
        colunas.remove(c)

# Insere as colunas novas logo após suas respectivas pistas
colunas.insert(idx_p1 + 1, 'AIDS_pista1')
colunas.insert(idx_p1 + 2, 'Aux_pista1')
colunas.insert(idx_p2 + 3, 'AIDS_pista2')  # +3 porque já inserimos 2 antes
colunas.insert(idx_p2 + 4, 'Aux_pista2')

# Reorganiza o DataFrame com as colunas na ordem desejada
aero_rotaer_com_aids = aero_rotaer_com_aids[colunas]


In [7]:

# Substitui NaN real
aero_rotaer_com_aids.fillna('-', inplace=True)

# Função para limpar strings com "nan", "nan 13", "N/A", etc.
def limpar_celula(valor):
    if isinstance(valor, str):
        # Remove qualquer ocorrência de "nan", "nan 13", "N/A", etc. (case-insensitive)
        valor_limpo = re.sub(r'(?i)nan\s*\d*', '', valor)
        valor_limpo = re.sub(r'(?i)n/?a|missing value', '', valor_limpo)

        # Remove barras duplicadas, espaços extras e deixa limpo
        valor_limpo = re.sub(r'\s*/\s*', ' / ', valor_limpo)  # normaliza barra
        valor_limpo = re.sub(r'\s+', ' ', valor_limpo).strip()  # tira espaços duplicados
        valor_limpo = re.sub(r'^/|/$', '', valor_limpo)  # remove barra no começo/fim

        return valor_limpo if valor_limpo else '-'
    elif pd.isna(valor):
        return '-'
    else:
        return valor

# Aplica a limpeza em todo o DataFrame
aero_rotaer_com_aids = aero_rotaer_com_aids.applymap(limpar_celula)



C:\Users\julia\AppData\Local\Temp\ipykernel_26408\399411891.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  aero_rotaer_com_aids = aero_rotaer_com_aids.applymap(limpar_celula)


In [8]:

aero_rotaer_com_aids.to_csv('output/aerodromos_BR_final.csv', index=False, encoding='utf-8')